#Capstone Project - The Battle of Neighborhoods (Week 1)

## 1. Business Problem

###*Miami, Florida*
Take a number of diverse cultures, add a strong dose of the arts and a splash of ocean water, and you have Miami. Looking at the fantastic art museums and the blossoming gastronomical scene, you might find it hard to believe that just a century ago, this colorful Floridian city was covered in swampland. Once developers rushed into the area, one of the most popular tourist destinations and spectacular city skylines in the country was born. Today, with South Beach before you and the Everglades behind you, you can walk through the bustling streets past historical homes with Spanish words and Caribbean music floating into your ears.

I went there beafore and really like that city(not during covid though, seems little bit dangerous:/). Anyway, Miami still one of the most popular travel destination in the US.

### *Opening of Hotel/Restaurant Shop*
Coming down to business problem, I'm going to discover where will be a good place to open a new hotel/resturanat in Miami.As it is a famous tourist spot, there is already lots of attention towards it. Due to the high competition between tourist industry, people need to locate hotel/resturant in place where more people are attracted and comfortable for a stay and a good meal. Futhermore, considering about the diversity people, I think the recipe of resturant should go with Italian, American, Mexican, and tpyical Cunjun food.

The challenge is to find a suitable location for opening a new hotel / restaurant attracted to all local and foreign people in the centre of all famous venues.

### *Expected / Interested Audience*
The travel type is quite different in Miami. Some people stay for couple of days or more. Also they find some place for hangout or a good meal. Their main focus might be belonging to stay somewhere near to reach venues. Apart from these set of people, students and working professionals are common audience here. So we may need to fascinate them all.



## 2. Data

We will be completely working on *Foursquare* data to explore and try to locate our new hotel where more venues like church, temples, beach, museums, memorials that are present nearby.

We will looking for midpoint area of venues to locate new hotel. Before that our major focus will be on all venues present in and around the core place of Miami.

Just a heads up on how many hotels are distributed now around Miami. We will perform some EDA on hotels & restaurants present in the tourist spot. On furthur notebook we will use Foursquare data to determine other venues as well.



In [1]:
# Import libraries
import numpy as np # data in a vectorized manner manipulation
import pandas as pd # data analsysis
import requests # HTTP library
from bs4 import BeautifulSoup # scraping library

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json # JSON files manipulation

from sklearn.cluster import KMeans # clustering algorithm

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

In [4]:
CLIENT_ID = '--' 
CLIENT_SECRET = '--' 
VERSION = '20210125' 
LIMIT = 500

In [6]:
address = "Miami, Florida"

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude =location.latitude   #8.079252 # location.latitude 
longitude =location.longitude #77.5499338 # location.longitude # 

kan='Miami location : {},{}'.format(latitude,longitude)
print(kan)

Miami location : 25.7741728,-80.19362


In [11]:
#Quering for hotel & restaurant

search_query = 'hotel'
search_query_res = 'restaurant'

radius = 1500
url_hotel = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url_restaurant = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query_res, radius, LIMIT)
#url

In [12]:
results_hotel = requests.get(url_hotel).json()
results_restaurant = requests.get(url_restaurant).json()
#results_hotel

In [13]:
# assign relevant part of JSON to venues
venues_hotel = results_hotel['response']['venues']
venues_restaurant = results_restaurant['response']['venues']

# tranform venues into a dataframe and merging both data
dataframe_hotel = json_normalize(venues_hotel)
dataframe_restaurant = json_normalize(venues_restaurant)

dataframe = pd.concat([dataframe_hotel,dataframe_restaurant])

print("There are {} restaurants and hotels at Miami".format(dataframe.shape[0]))

There are 100 restaurants and hotels at Miami


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


In [14]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

  
# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

#dataframe_filtered
hotels_df=dataframe_filtered[['name','categories','distance','lat','lng','id']]
hotels_df.head()

,name,categories,distance,lat,lng,id
0,Kimpton EPIC Hotel,Hotel,572,25.770571,-80.189539,4a9f2a6af964a520c43c20e3
1,Hotel Beaux Arts Miami,Hotel,526,25.771239,-80.189500,4d6ecdeb7c726dcb9cb4811d
2,YVE Hotel Miami,Hotel,570,25.775670,-80.188180,5842240b45005e59fe0281a4
3,SLS Brickell Hotel & Residence,Hotel,1447,25.761170,-80.193651,5997438b12138410b7334d89
4,SP+ Parking @ Hyatt Regency Miami Hotel,Parking,468,25.771394,-80.190105,573a2f6b498e30fec8b19552


In [18]:
hotels_map = folium.Map(location=[latitude, longitude], zoom_start=16) # generate map centred around the Miami

# add a red circle marker to represent the core location of Miami
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Miami',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(hotels_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(hotels_df.lat, hotels_df.lng, hotels_df.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(hotels_map)

# display map
hotels_map